TEST 1


In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO('best.pt')

cap = cv2.VideoCapture('15sec_input_720p.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference
    results = model(frame)

    # Visualize results on the frame
    annotated_frame = results[0].plot()

    # Show the annotated frame
    cv2.imshow('Player Detection', annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Test 2

In [ ]:
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load your YOLOv11 model
model = YOLO('best.pt')

# Initialize DeepSORT Tracker
tracker = DeepSort(max_age=30, n_init=3, nms_max_overlap=1.0)

# Open video file
cap = cv2.VideoCapture('15sec_input_720p.mp4')

# Get video properties for saving
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
out = cv2.VideoWriter('output_with_tracking.mp4', fourcc, fps, (frame_width, frame_height))

# Calculate delay for real-time-like playback
delay = int(1000 / fps)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection
    results = model(frame)

    detections = []
    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss = r.boxes.cls.cpu().numpy()

        for box, conf, cls in zip(boxes, confs, clss):
            if int(cls) == 0 and conf > 0.5:  # Assuming class 0 is player
                x1, y1, x2, y2 = box
                detections.append(([x1, y1, x2, y2], conf, 'player'))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw results
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'Player {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Save the annotated frame
    out.write(frame)

    # Show the annotated frame
    cv2.imshow('Player Tracking', frame)

    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


TEST 3

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from scipy.spatial import distance as dist

# Centroid Tracker Class
class CentroidTracker:
    def __init__(self, max_disappeared=40):
        self.nextObjectID = 0
        self.objects = {}  # ID -> centroid
        self.disappeared = {}
        self.max_disappeared = max_disappeared

    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.max_disappeared:
                    self.deregister(objectID)
            return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        for (i, (x1, y1, x2, y2)) in enumerate(rects):
            cX = int((x1 + x2) / 2.0)
            cY = int((y1 + y2) / 2.0)
            inputCentroids[i] = (cX, cY)

        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                self.objects[objectIDs[row]] = inputCentroids[col]
                self.disappeared[objectIDs[row]] = 0
                usedRows.add(row)
                usedCols.add(col)

            unusedRows = set(range(0, D.shape[0])) - usedRows
            unusedCols = set(range(0, D.shape[1])) - usedCols

            for row in unusedRows:
                objectID = objectIDs[row]
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.max_disappeared:
                    self.deregister(objectID)

            for col in unusedCols:
                self.register(inputCentroids[col])

        return self.objects
    
    # Load YOLOv11 model
model = YOLO('best.pt')

# Initialize tracker
ct = CentroidTracker()

# Open video
cap = cv2.VideoCapture('15sec_input_720p.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference
    results = model(frame)

    rects = []
    player_data = []

    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss = r.boxes.cls.cpu().numpy()

        for box, conf, cls in zip(boxes, confs, clss):
            print(f"Detected class: {int(cls)}, confidence: {conf}")

            if int(cls) == 1 and conf > 0.5:  # Replace 1 with correct player class if needed
                x1, y1, x2, y2 = box
                cX = int((x1 + x2) / 2.0)
                cY = int((y1 + y2) / 2.0)
                player_data.append(((cX, cY), (x1, y1, x2, y2)))

    centroids = [d[0] for d in player_data]
    objects = ct.update(centroids) 

    # Draw tracked objects with consistent IDs
    for (objectID, centroid) in objects.items():
    # Find matching bounding box
        for (c, (x1, y1, x2, y2)) in player_data:
            if np.linalg.norm(np.array(centroid) - np.array(c)) < 30:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame, f'Player {objectID}', (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Player Detection with Consistent IDs', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

TEST 4

In [ ]:
from ultralytics import YOLO
import cv2

# Load the fine-tuned YOLOv8 model
model = YOLO('best.pt') 

# Open the video file
video_path = '15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video {video_path}")
    exit()

while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    if not ret:
        print("End of video or error reading frame.")
        break

    # --- KEY CHANGE: Use model.track() instead of model() ---
    # 'persist=True' tells the tracker to remember identities between frames
    results = model.track(frame, persist=True)

    # Check if any tracks were detected
    if results[0].boxes.id is not None:
        # Get the bounding boxes, track IDs, and confidence scores
        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        track_ids = results[0].boxes.id.cpu().numpy().astype(int)
        confidences = results[0].boxes.conf.cpu().numpy()

        # Loop through each detected player
        for box, track_id, conf in zip(boxes, track_ids, confidences):
            # Draw the bounding box
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            # Create the label text with Player ID and confidence
            label = f"Player ID: {track_id}"
            
            # Position the label text above the bounding box
            label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(frame, (box[0], box[1] - label_size[1] - 5), (box[0] + label_size[0], box[1]), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, label, (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Show the annotated frame
    cv2.imshow('Player Tracking', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


TEST 4 PART 2

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# --- COLOR/TEAM DEFINITIONS ---
# Define HSV color ranges for Red and Sky Blue

# Red wraps around the 0-180 hue scale
LOWER_RED_1 = np.array([0, 120, 70])
UPPER_RED_1 = np.array([10, 255, 255])
LOWER_RED_2 = np.array([170, 120, 70])
UPPER_RED_2 = np.array([180, 255, 255])

# UPDATED: HSV range for Sky Blue (more towards cyan, allows for lower saturation)
LOWER_SKY_BLUE = np.array([90, 70, 50])
UPPER_SKY_BLUE = np.array([120, 255, 255])

# BGR colors for drawing boxes
RED_COLOR_BGR = (0, 0, 255)
# UPDATED: A lighter BGR color to represent Sky Blue
SKY_BLUE_COLOR_BGR = (235, 206, 135) 
GREEN_COLOR_BGR = (0, 255, 0) # For undetermined

def get_player_team(player_roi):
    """
    Analyzes a player's Region of Interest (ROI) to determine their team color.
    
    Args:
        player_roi (np.array): The cropped image of the player.
        
    Returns:
        tuple: A tuple containing the team name (str) and the BGR color for the bounding box.
    """
    if player_roi.size == 0:
        return "Undetermined", GREEN_COLOR_BGR

    # Convert ROI to HSV color space
    hsv_roi = cv2.cvtColor(player_roi, cv2.COLOR_BGR2HSV)

    # Create masks for red and sky blue colors
    red_mask1 = cv2.inRange(hsv_roi, LOWER_RED_1, UPPER_RED_1)
    red_mask2 = cv2.inRange(hsv_roi, LOWER_RED_2, UPPER_RED_2)
    red_mask = cv2.bitwise_or(red_mask1, red_mask2)
    
    # UPDATED: Use the new sky blue mask
    sky_blue_mask = cv2.inRange(hsv_roi, LOWER_SKY_BLUE, UPPER_SKY_BLUE)

    # Count the number of red and sky blue pixels
    red_pixel_count = cv2.countNonZero(red_mask)
    sky_blue_pixel_count = cv2.countNonZero(sky_blue_mask)

    # Determine team based on which color is more dominant
    if red_pixel_count > sky_blue_pixel_count and red_pixel_count > 10: # Min pixel threshold
        return "Red Team", RED_COLOR_BGR
    elif sky_blue_pixel_count > red_pixel_count and sky_blue_pixel_count > 10:
        # UPDATED: Return "Sky Blue Team" and the new color
        return "Sky Blue Team", SKY_BLUE_COLOR_BGR
    else:
        return "Undetermined", GREEN_COLOR_BGR

# --- MAIN SCRIPT ---

# Load the fine-tuned YOLOv8 model
model = YOLO('best.pt') 

# Open the video file
video_path = '15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video {video_path}")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True)

    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        track_ids = results[0].boxes.id.cpu().numpy().astype(int)

        for box, track_id in zip(boxes, track_ids):
            x1, y1, x2, y2 = box
            
            # Crop the player's torso for color analysis
            torso_height = int((y2 - y1) * 0.5)
            player_torso_roi = frame[y1:y1 + torso_height, x1:x2]
            
            team_name, box_color = get_player_team(player_torso_roi)
            
            # Draw the bounding box with the team's color
            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)

            # Create the label text with Team and Player ID
            label = f"{team_name} ID: {track_id}"
            
            # Position the label text
            label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), box_color, cv2.FILLED)
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow('Player Tracking and Team Segregation', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Test 5 using Deepsort

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# --- COLOR/TEAM DEFINITIONS ---
LOWER_RED_1 = np.array([0, 120, 70])
UPPER_RED_1 = np.array([10, 255, 255])
LOWER_RED_2 = np.array([170, 120, 70])
UPPER_RED_2 = np.array([180, 255, 255])
LOWER_SKY_BLUE = np.array([90, 70, 50])
UPPER_SKY_BLUE = np.array([120, 255, 255])

RED_COLOR_BGR = (0, 0, 255)
SKY_BLUE_COLOR_BGR = (235, 206, 135)
GREEN_COLOR_BGR = (0, 255, 0)


def get_player_team(player_roi):
    if player_roi.size == 0:
        return "Undetermined", GREEN_COLOR_BGR

    hsv_roi = cv2.cvtColor(player_roi, cv2.COLOR_BGR2HSV)
    red_mask1 = cv2.inRange(hsv_roi, LOWER_RED_1, UPPER_RED_1)
    red_mask2 = cv2.inRange(hsv_roi, LOWER_RED_2, UPPER_RED_2)
    red_mask = cv2.bitwise_or(red_mask1, red_mask2)
    sky_blue_mask = cv2.inRange(hsv_roi, LOWER_SKY_BLUE, UPPER_SKY_BLUE)

    red_pixel_count = cv2.countNonZero(red_mask)
    sky_blue_pixel_count = cv2.countNonZero(sky_blue_mask)

    if red_pixel_count > sky_blue_pixel_count and red_pixel_count > 10:
        return "Red Team", RED_COLOR_BGR
    elif sky_blue_pixel_count > red_pixel_count and sky_blue_pixel_count > 10:
        return "Blue Team", SKY_BLUE_COLOR_BGR
    else:
        return "Undetermined", GREEN_COLOR_BGR


# --- MAIN SCRIPT ---

model = YOLO('best.pt')
tracker = DeepSort(max_age=30, n_init=2, nms_max_overlap=1.0)

cap = cv2.VideoCapture('15sec_input_720p.mp4')
if not cap.isOpened():
    print("Error: Could not open video")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    detections = []

    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss = r.boxes.cls.cpu().numpy()

        for box, conf, cls in zip(boxes, confs, clss):
            if int(cls) == 1 and conf > 0.5:  # Assuming class 1 is player
                x1, y1, x2, y2 = box.astype(int)
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'player'))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        torso_height = int((y2 - y1) * 0.5)
        player_torso_roi = frame[y1:y1 + torso_height, x1:x2]

        team_name, box_color = get_player_team(player_torso_roi)

        cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)
        label = f"{team_name} ID: {track_id}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), box_color, cv2.FILLED)
        cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow('Player Tracking with DeepSORT & Team Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


TEST 6 

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# --- COLOR/TEAM DEFINITIONS (Same as before) ---
LOWER_RED_1 = np.array([0, 120, 70])
UPPER_RED_1 = np.array([10, 255, 255])
LOWER_RED_2 = np.array([170, 120, 70])
UPPER_RED_2 = np.array([180, 255, 255])
LOWER_SKY_BLUE = np.array([90, 70, 50])
UPPER_SKY_BLUE = np.array([120, 255, 255])
RED_COLOR_BGR = (0, 0, 255)
SKY_BLUE_COLOR_BGR = (235, 206, 135)
GREEN_COLOR_BGR = (0, 255, 0)

# --- NEW: APPEARANCE FEATURE FUNCTION ---
def create_color_histogram(roi):
    """Creates a 3D color histogram from the player's torso ROI."""
    if roi.size == 0:
        return None
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    # Use 3 channels (H, S, V), with more bins for Hue for better color distinction
    hist = cv2.calcHist([hsv_roi], [0, 1, 2], None, [8, 3, 3], [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)
    return hist

def get_player_team(player_roi):
    # This function remains the same as before
    if player_roi.size == 0: return "Undetermined", GREEN_COLOR_BGR
    hsv_roi = cv2.cvtColor(player_roi, cv2.COLOR_BGR2HSV)
    red_mask1 = cv2.inRange(hsv_roi, LOWER_RED_1, UPPER_RED_1)
    red_mask2 = cv2.inRange(hsv_roi, LOWER_RED_2, UPPER_RED_2)
    red_mask = cv2.bitwise_or(red_mask1, red_mask2)
    sky_blue_mask = cv2.inRange(hsv_roi, LOWER_SKY_BLUE, UPPER_SKY_BLUE)
    red_pixel_count = cv2.countNonZero(red_mask)
    sky_blue_pixel_count = cv2.countNonZero(sky_blue_mask)
    if red_pixel_count > sky_blue_pixel_count and red_pixel_count > 10:
        return "Red Team", RED_COLOR_BGR
    elif sky_blue_pixel_count > red_pixel_count and sky_blue_pixel_count > 10:
        return "Sky Blue Team", SKY_BLUE_COLOR_BGR
    else:
        return "Undetermined", GREEN_COLOR_BGR

# --- MAIN SCRIPT ---
model = YOLO('best.pt')
video_path = '15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

# --- NEW: TRACK MANAGEMENT DICTIONARIES ---
# active_tracks stores current data for visible players
# { track_id: {"box": [...], "team": "...", "hist": ..., "frames_since_seen": 0} }
active_tracks = {}
# disappeared_tracks stores data for players who have left the frame
# { track_id: {"box": [...], "team": "...", "hist": ..., "frames_since_seen": 0} }
disappeared_tracks = {}

# Re-ID parameters
MAX_FRAMES_TO_DISAPPEAR = 30  # How many frames to keep a disappeared track in memory
HIST_COMPARISON_THRESHOLD = 0.6  # How similar histograms must be to be a match

frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_num += 1

    results = model.track(frame, persist=True)
    
    current_track_ids = set()
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        track_ids = results[0].boxes.id.cpu().numpy().astype(int)

        # --- PROCESS CURRENTLY VISIBLE TRACKS ---
        for box, track_id in zip(boxes, track_ids):
            current_track_ids.add(track_id)
            
            # Crop player's torso for analysis
            x1, y1, x2, y2 = box
            torso_height = int((y2 - y1) * 0.5)
            player_torso_roi = frame[y1:y1 + torso_height, x1:x2]

            # --- RE-IDENTIFICATION LOGIC ---
            if track_id not in active_tracks:
                # This is a NEW track ID. Check if it matches a disappeared player.
                is_new_player = True
                new_hist = create_color_histogram(player_torso_roi)
                if new_hist is not None:
                    best_match_id = -1
                    max_similarity = -1
                    
                    # Compare with all disappeared tracks
                    for d_id, d_data in list(disappeared_tracks.items()):
                        similarity = cv2.compareHist(new_hist, d_data["hist"], cv2.HISTCMP_CORREL)
                        if similarity > HIST_COMPARISON_THRESHOLD and similarity > max_similarity:
                            max_similarity = similarity
                            best_match_id = d_id
                    
                    if best_match_id != -1:
                        # Match found! Re-assign the old ID.
                        print(f"Re-identified: New ID {track_id} -> Old ID {best_match_id}")
                        active_tracks[best_match_id] = disappeared_tracks[best_match_id]
                        active_tracks[best_match_id]["box"] = box # Update box
                        active_tracks[best_match_id]["frames_since_seen"] = 0 # Reset counter
                        del disappeared_tracks[best_match_id] # Remove from disappeared
                        current_track_ids.add(best_match_id) # Add to current set
                        is_new_player = False

                if is_new_player:
                    # No match found, it's a genuinely new player.
                    team_name, _ = get_player_team(player_torso_roi)
                    active_tracks[track_id] = {
                        "box": box,
                        "team": team_name,
                        "hist": new_hist,
                        "frames_since_seen": 0
                    }
            else:
                # This is an existing, tracked player. Update their data.
                active_tracks[track_id]["box"] = box
                active_tracks[track_id]["frames_since_seen"] = 0

    # --- MANAGE DISAPPEARED TRACKS ---
    # Increment 'frames_since_seen' for all active tracks not seen in this frame
    disappeared_ids = set(active_tracks.keys()) - current_track_ids
    for track_id in disappeared_ids:
        active_tracks[track_id]["frames_since_seen"] += 1
        # If a player has been gone for too long, move them to disappeared_tracks
        if active_tracks[track_id]["frames_since_seen"] > 5: # Small buffer before moving
            disappeared_tracks[track_id] = active_tracks.pop(track_id)
            print(f"Track {track_id} moved to disappeared.")

    # Remove tracks that have been disappeared for too long
    for track_id in list(disappeared_tracks.keys()):
        if disappeared_tracks[track_id]["frames_since_seen"] > MAX_FRAMES_TO_DISAPPEAR:
            print(f"Track {track_id} removed permanently from memory.")
            del disappeared_tracks[track_id]

    # --- VISUALIZATION ---
    for track_id, data in active_tracks.items():
        if data["frames_since_seen"] == 0: # Only draw active players
            box = data['box']
            team_name, box_color = get_player_team(frame[box[1]:box[3], box[0]:box[2]])
            label = f"{team_name} ID: {track_id}"
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), box_color, 2)
            label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame, (box[0], box[1] - label_size[1] - 10), (box[0] + label_size[0], box[1]), box_color, cv2.FILLED)
            cv2.putText(frame, label, (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
    cv2.imshow('Player Re-Identification', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()